## Imports

In [85]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [86]:
df = pd.read_csv('redfin_2023-08-07-09-19-29.csv')

## Data Clean

In [87]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [88]:
df = df.dropna(subset=['SOLD DATE'])

In [89]:
# Define list of desired months (excluding current month)
desired_months = ['July']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [90]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    841
Name: count, dtype: int64
-------
$/SQUARE FEET
False    834
True       7
Name: count, dtype: int64
-------
YEAR BUILT
False    841
Name: count, dtype: int64
-------


In [91]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [92]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [93]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
185,PAST SALE,July-25-2023,Condo/Co-op,9917 W Okeechobee Rd Unit 4-305,Hialeah Gardens,FL,33016.0,250.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/9917-W-Okeec...,MARMLS,A11379050,N,Y,25.866333,-80.334622
422,PAST SALE,July-5-2023,Condo/Co-op,14951 Royal Oaks Ln #2306,North Miami,FL,33181.0,400.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/14951-Ro...,MARMLS,A11365864,N,Y,25.913867,-80.151668
631,PAST SALE,July-18-2023,Condo/Co-op,290 174th St #308,Sunny Isles Beach,FL,33160.0,44500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/29...,MARMLS,A11332103,N,Y,25.936045,-80.130062
667,PAST SALE,July-21-2023,Condo/Co-op,13480 NE 6th Ave #107,North Miami,FL,33161.0,67000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13480-NE...,MARMLS,A11332382,N,Y,25.898940,-80.187168
809,PAST SALE,July-26-2023,Condo/Co-op,13480 NE 6th Ave #216,North Miami,FL,33161.0,69885.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13480-NE...,MARMLS,A11267636,N,Y,25.898940,-80.187168
8,PAST SALE,July-25-2023,Condo/Co-op,13480 NE 6th Ave #303,North Miami,FL,33161.0,70000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13480-NE...,MARMLS,A11405950,N,Y,25.898940,-80.187168
394,PAST SALE,July-25-2023,Condo/Co-op,13480 NE 6th Ave #304,North Miami,FL,33161.0,75000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13480-NE...,MARMLS,A11376319,N,Y,25.898940,-80.187168
786,PAST SALE,July-18-2023,Condo/Co-op,18700 NE 3rd Ct #620,Miami,FL,33179.0,90537.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/18700-NE-3rd-C...,MARMLS,A11296791,N,Y,25.947261,-80.194442
557,PAST SALE,July-28-2023,Condo/Co-op,55 NW 204th St #15,Miami Gardens,FL,33169.0,93000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/55-NW-204th-St...,Beaches MLS,F10376247,N,Y,25.963215,-80.203585
473,PAST SALE,July-14-2023,Condo/Co-op,1351 NE Miami Gardens Dr Unit 1122E,Miami,FL,33179.0,135000.0,1.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/13...,MARMLS,A11358458,N,Y,25.945855,-80.174575


In [94]:
print(df_filtered['URL'].iloc[382])

https://www.redfin.com/FL/North-Bay-Village/7501-E-Treasure-Dr-33141/unit-5M/home/43344225


In [95]:
# Correct the prices, if needed
df_filtered.at[185,'PRICE']=(250000)
df_filtered.at[422,'PRICE']=(400000)
# df_filtered.at[1377,'PRICE']=(710000)
# df_filtered.at[1115,'PRICE']=(445000)

In [96]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [97]:
# # Corrections, if needed
df_filtered.at[185,'$/SQUARE FEET']=(250000/1190)
df_filtered.at[422,'$/SQUARE FEET']=(400000/1730)
# df_filtered.at[1377,'$/SQUARE FEET']=(710000/1055)
# df_filtered.at[1115,'$/SQUARE FEET']=(445000/850)

In [98]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
631,44500.0,290 174th St #308,Sunny Isles Beach,36.0
394,75000.0,13480 NE 6th Ave #304,North Miami,82.0
809,69885.0,13480 NE 6th Ave #216,North Miami,82.0
8,70000.0,13480 NE 6th Ave #303,North Miami,82.0
667,67000.0,13480 NE 6th Ave #107,North Miami,92.0
456,138500.0,1770 NE 191st St Unit 116-1,Miami,105.0
786,90537.0,18700 NE 3rd Ct #620,Miami,107.0
752,180000.0,1750 NE 191st St Unit 426-4,Miami,137.0
688,172000.0,484 NW 165th St Rd Unit A-212,Miami,146.0
181,174000.0,488 NW 165 Th Unit B409,Miami,148.0


In [99]:
print(df_filtered.URL.iloc[760])

https://www.redfin.com/FL/Miami-Beach/5970-Indian-Creek-Dr-33140/unit-4/home/43318404


In [100]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [101]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [102]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [103]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [104]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [105]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [106]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"July 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [107]:
m.save('index.html')

## Summary Info

In [108]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [109]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [110]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [118]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Sunny-Isles-Beach/290-174th-St-33160/unit-308/home/43243768


In [112]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
7400 OCEANSIDE AT FISHER, 7463 Fisher Island Dr #7463 Miami Beach | Price $24,175,000 | $3,177 psf | Year built: 2002
Least Expensive
WINSTON TOWERS 700 CONDO, 290 174th St #308 Sunny Isles Beach | Price $44,500 | $36 psf | Year built: 1980
Highest Price Per Square Foot
7400 OCEANSIDE AT FISHER, 7463 Fisher Island Dr #7463 Miami Beach | Price $24,175,000 | $3,177 psf | Year built: 2002
Lowest Price Per Square Foot
WINSTON TOWERS 700 CONDO, 290 174th St #308 Sunny Isles Beach | Price $44,500 | $36 psf | Year built: 1980
Newest
MISSONI BAIA, 700 NE 26th Ter #804 Miami | Price $680,000 | $nan psf | Year built: 2023
Oldest
PALM GARDEN CONDO, 760 Meridian Ave #1 Miami Beach | Price $735,000 | $721 psf | Year built: 1923


## Time on Market Calculator

In [113]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [119]:
from datetime import datetime, timedelta

date1 = datetime(2022, 1, 24) ## List (Earlier) date
date2 = datetime(2023, 7, 18) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

540


## Map URL Snagger

In [115]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [116]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_month_ending_july_2023 copy


## Get Summary Data

In [117]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 841
--------
Total sale price: $592,548,377
--------
Median sale price: $415,000
--------
Max sale price: $24,175,000
--------
Min sale price: $44,500
------------------------------------------------
PSF INFO
Max price per square foot: $3,177
--------
Min price per square foot: $36
--------
Median price per square foot: $435
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1923.0
----------
Average building age: 1988.6052318668253
